<table style="width: 100%">
    <tr>
    <td style="width: 60%; text-align: center">
<h1>Data Analysis & Explanation</h1>
     </tr>
</table>

In [ ]:
# Importation
from util import dataReader, dataTransform_date, quoted_spread
import pandas as pd
import numpy as np
import sys

#plotly display error handling
import plotly.io as pio
pio.renderers.default = "iframe"

import plotly.express as px #interactive graphic
import plotly.figure_factory as ff

# Avoiding warnings
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

##### Data collection and engineering

In [ ]:
rfq_raw = dataReader("input_data/rfqs.csv")

In [ ]:
df_rfq = rfq_raw.copy(deep=True)

In [ ]:
df_rfq.head()

In [ ]:
df_rfq.drop(["Unnamed: 0"], axis=1, inplace=True)

In [ ]:
df_rfq = dataTransform_date(df_rfq, 'date_time')

In [ ]:
df_rfq.head()

In [ ]:
df_rfq = quoted_spread(df_rfq)

In [ ]:
df_rfq.head()

##### Data visualization

In [ ]:
df_instrument = pd.DataFrame(df_rfq.groupby(['instrument'])['date_time'].count()).reset_index()
df_client = pd.DataFrame(df_rfq.groupby(['client'])['date_time'].count()).reset_index()
df_timing = pd.DataFrame(df_rfq.groupby(['month'])['date_time'].count()).reset_index()

df_spread_month = pd.DataFrame(df_rfq.groupby(['month'])['quoted_spread'].mean()).reset_index()
df_spread_instrument = pd.DataFrame(df_rfq.groupby(['instrument'])['quoted_spread'].mean()).reset_index()

In [ ]:
fig = px.histogram(df_instrument, x='instrument', y='date_time', title="Main instrument(s) requested by clients")
fig.update_layout(plot_bgcolor='white')
fig.show()

In [ ]:
df_instrument_t1 = pd.DataFrame(df_rfq.groupby(['instrument', 'won'])['date_time'].count()).reset_index()

In [ ]:
prop_win = df_instrument_t1.groupby('instrument').apply(
        lambda group: pd.Series({
        "losses": group.loc[group["won"] == 0, "date_time"].sum(),
        "wins": group.loc[group["won"] == 1, "date_time"].sum(),
    })
).reset_index()

prop_win["win_proportion"] = prop_win["wins"] / (prop_win["wins"] + prop_win["losses"])
prop_win["loss_proportion"] = prop_win["losses"] / (prop_win["wins"] + prop_win["losses"])

In [ ]:
fig = px.histogram(prop_win, x='instrument', y='win_proportion', title="ratio gain")
fig.update_layout(plot_bgcolor='white')
fig.show()

In [ ]:
df_instrument_t2 = pd.DataFrame(df_rfq.groupby(['instrument', 'won'])['quoted_spread'].mean()).reset_index()

In [ ]:
fig = px.histogram(df_instrument_t2, x='instrument', y='quoted_spread', color='won', title="Spread per instrument")
fig.update_layout(plot_bgcolor='white')
fig.show()

In [ ]:
fig = px.line(df_timing, x='month', y='date_time', title="Time trend requested quote")
fig.update_layout(plot_bgcolor='white')
fig.show()

In [ ]:
fig = px.line(df_spread_month, x='month', y='quoted_spread', title="Time trend spread")
fig.update_layout(plot_bgcolor='white')
fig.show()

In [ ]:
fig = px.histogram(df_spread_instrument, x='instrument', y='quoted_spread', title="Instrument trend spread")
fig.update_layout(plot_bgcolor='white')
fig.show()

##### Clustering

In [ ]:
data_model = df_rfq.drop(columns=['date_time'])

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
class_transformer = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0, 1])], remainder='passthrough') #We applied the encoder only for instrument and client
data_model = class_transformer.fit_transform(data_model)

In [ ]:
from sklearn.cluster import KMeans

nb_clusters = range(1,11)
inertia = []

#recherche de clusters optimal
for i in nb_clusters:
    kmeans = KMeans(n_clusters=i, random_state=2)
    kmeans = kmeans.fit(data_model)
    u = kmeans.inertia_
    inertia.append(u)
    print("L'inertie pour :", i, "clusters est:", u)

In [ ]:
x = np.arange(len(nb_clusters))
y = inertia
df_plot_inertia = pd.DataFrame([x, y]).T

df_plot_inertia.rename(columns={0: 'Cluster', 1: 'Inertia'}, inplace=True)
# df_plot_inertia.columns

In [ ]:
fig = px.line(df_plot_inertia, x='Cluster', y="Inertia", title='Visualization inertia per cluster')
fig.update_layout(plot_bgcolor='white')
fig.show()

In [ ]:
# Application with 4 clusters
kmeans = KMeans(n_clusters=4, random_state=2)
kmeans = kmeans.fit(data_model)

df_rfq["class"] = kmeans.labels_

class_1 = df_rfq[df_rfq["class"]==0]
class_2 = df_rfq[df_rfq["class"]==1]
class_3 = df_rfq[df_rfq["class"]==2]
class_4 = df_rfq[df_rfq["class"]==3]

#### Data analysis and visualization for cluster 1

In [ ]:
class_1.quoted_spread.mean()

In [ ]:
df_instrument1 = pd.DataFrame(class_1.groupby(['instrument'])['date_time'].count()).reset_index()
df_client1 = pd.DataFrame(class_1.groupby(['client'])['date_time'].count()).reset_index()
df_timing1 = pd.DataFrame(class_1.groupby(['month'])['date_time'].count()).reset_index()

In [ ]:
fig = px.histogram(df_instrument1, x='instrument', y='date_time', title="Main instrument(s) in cluster 1")
fig.update_layout(plot_bgcolor='white')
fig.show()

In [ ]:
fig = px.histogram(df_client1, x='client', y='date_time', title="Main client(s) in cluster 1")
fig.update_layout(plot_bgcolor='white')
fig.show()

In [ ]:
fig = px.line(df_timing1, x='month', y='date_time', title="Time trend in cluster 1")
fig.update_layout(plot_bgcolor='white')
fig.show()

In [ ]:
df_spread_month1 = pd.DataFrame(class_1.groupby(['month'])['quoted_spread'].mean()).reset_index()
df_spread_instrument1 = pd.DataFrame(class_1.groupby(['instrument'])['quoted_spread'].mean()).reset_index()

In [ ]:
fig = px.line(df_spread_month1, x='month', y='quoted_spread', title="Spread per month in cluster 1")
fig.update_layout(plot_bgcolor='white')
fig.show()

In [ ]:
fig = px.histogram(df_spread_instrument1, x='instrument', y='quoted_spread', title="Spread per instrument in cluster 1")
fig.update_layout(plot_bgcolor='white')
fig.show()

#### Data analysis and visualization for cluster 2

In [ ]:
class_2.quoted_spread.mean()

In [ ]:
df_instrument2 = pd.DataFrame(class_2.groupby(['instrument'])['date_time'].count()).reset_index()
df_client2 = pd.DataFrame(class_2.groupby(['client'])['date_time'].count()).reset_index()
df_timing2 = pd.DataFrame(class_2.groupby(['month'])['date_time'].count()).reset_index()

In [ ]:
fig = px.histogram(df_instrument2, x='instrument', y='date_time', title="Main instrument(s) in cluster 2")
fig.update_layout(plot_bgcolor='white')
fig.show()

In [ ]:
fig = px.histogram(df_client2, x='client', y='date_time', title="Main client(s) in cluster 2")
fig.update_layout(plot_bgcolor='white')
fig.show()

In [ ]:
fig = px.line(df_timing2, x='month', y='date_time', title="Number of deal requested by clients per month in cluster 3")
fig.update_layout(plot_bgcolor='white')
fig.show()

In [ ]:
df_spread_month2 = pd.DataFrame(class_2.groupby(['month'])['quoted_spread'].mean()).reset_index()
df_spread_instrument2 = pd.DataFrame(class_2.groupby(['instrument'])['quoted_spread'].mean()).reset_index()

In [ ]:
fig = px.line(df_spread_month2, x='month', y='quoted_spread', title="Spread per month in cluster 2")
fig.update_layout(plot_bgcolor='white')
fig.show()

In [ ]:
fig = px.histogram(df_spread_instrument2, x='instrument', y='quoted_spread', title="Spread per instrument in cluster 2")
fig.update_layout(plot_bgcolor='white')
fig.show()

#### Data analysis and visualization for cluster 3

In [ ]:
class_3.quoted_spread.mean()

In [ ]:
df_instrument3 = pd.DataFrame(class_3.groupby(['instrument'])['date_time'].count()).reset_index()
df_client3 = pd.DataFrame(class_3.groupby(['client'])['date_time'].count()).reset_index()
df_timing3 = pd.DataFrame(class_3.groupby(['month'])['date_time'].count()).reset_index()

In [ ]:
fig = px.histogram(df_instrument3, x='instrument', y='date_time', title="Main instrument(s) in cluster 3")
fig.update_layout(plot_bgcolor='white')
fig.show()

In [ ]:
fig = px.histogram(df_client3, x='client', y='date_time', title="Main client(s) in cluster 3")
fig.update_layout(plot_bgcolor='white')
fig.show()

In [ ]:
fig = px.line(df_timing3, x='month', y='date_time', title="Number of deal requested by clients per month in cluster 3")
fig.update_layout(plot_bgcolor='white')
fig.show()

In [ ]:
df_spread_month3 = pd.DataFrame(class_3.groupby(['month'])['quoted_spread'].mean()).reset_index()
df_spread_instrument3 = pd.DataFrame(class_3.groupby(['instrument'])['quoted_spread'].mean()).reset_index()

In [ ]:
fig = px.line(df_spread_month3, x='month', y='quoted_spread', title="Spread per month in cluster 3")
fig.update_layout(plot_bgcolor='white')
fig.show()

In [ ]:
fig = px.histogram(df_spread_instrument3, x='instrument', y='quoted_spread', title="Spread per instrument in cluster 3")
fig.update_layout(plot_bgcolor='white')
fig.show()

#### Data analysis and visualization for cluster 4

In [ ]:
class_4.quoted_spread.mean()

In [ ]:
df_instrument4 = pd.DataFrame(class_4.groupby(['instrument'])['date_time'].count()).reset_index()
df_client4 = pd.DataFrame(class_4.groupby(['client'])['date_time'].count()).reset_index()
df_timing4 = pd.DataFrame(class_4.groupby(['month'])['date_time'].count()).reset_index()

In [ ]:
fig = px.histogram(df_instrument4, x='instrument', y='date_time', title="Main instrument(s) in cluster 4")
fig.update_layout(plot_bgcolor='white')
fig.show()

In [ ]:
fig = px.histogram(df_client4, x='client', y='date_time', title="Main client(s) in cluster 4")
fig.update_layout(plot_bgcolor='white')
fig.show()

In [ ]:
fig = px.line(df_timing4, x='month', y='date_time', title="Number of deal requested by clients per month in cluster 4")
fig.update_layout(plot_bgcolor='white')
fig.show()

In [ ]:
df_spread_month4 = pd.DataFrame(class_4.groupby(['month'])['quoted_spread'].mean()).reset_index()
df_spread_instrument4 = pd.DataFrame(class_4.groupby(['instrument'])['quoted_spread'].mean()).reset_index()

In [ ]:
fig = px.line(df_spread_month4, x='month', y='quoted_spread', title="Spread per month in cluster 4")
fig.update_layout(plot_bgcolor='white')
fig.show()

In [ ]:
fig = px.histogram(df_spread_instrument4, x='instrument', y='quoted_spread', title="Spread per instrument in cluster 4")
fig.update_layout(plot_bgcolor='white')
fig.show()

In [ ]:
ration_winning_1 = len(class_1[class_1['won']==1]) / len(class_1)
ration_winning_2 = len(class_2[class_2['won']==1]) / len(class_2)
ration_winning_3 = len(class_3[class_3['won']==1]) / len(class_3)
ration_winning_4 = len(class_4[class_4['won']==1]) / len(class_4)

In [ ]:
ration_winning_1

In [ ]:
ration_winning_2

In [ ]:
ration_winning_3

In [ ]:
ration_winning_4